In [152]:
import pandas as pd
import re
from artists_88rising import artists

In [153]:
df_88R = pd.read_csv("../88RisingArtists/DataSets/final88data.csv")
df_88R.drop('ID', axis=1, inplace=True)
df_88R.head()

,song name,album,artist,featured,release_date,length(ms),popularity,acousticness,danceability,energy,liveness,tempo,valence
0,The Weekend,The Weekend,88rising,"['88rising', 'BIBI']",2021-10-14,167661,69,0.06200,0.784,0.521,0.0995,101.491,0.817
1,LAW (Prod. Czaer),Street Man Fighter Original Vol.3 (Mission by ...,Various Artists,"['Yoon Mirae', 'BIBI']",2022-09-06,189373,71,0.00857,0.756,0.900,0.2370,128.054,0.550
2,Best Lover,Head In The Clouds Forever,88rising,"['88rising', 'BIBI']",2022-04-16,152910,66,0.26800,0.740,0.663,0.1210,82.004,0.519
3,Animal Farm,Animal Farm,BIBI,['BIBI'],2022-09-27,203945,69,0.01780,0.617,0.570,0.5500,107.971,0.375
4,KAZINO,KAZINO,BIBI,['BIBI'],2020-04-29,185946,65,0.13900,0.625,0.686,0.1100,133.041,0.263


In [67]:
len(df_88R)

451

In [68]:
type(df_88R['featured'][0])

str

Featured column is stored as a string, but needs to be converted into a list.

In [154]:
df_88R['featured'] = df_88R['featured'].apply(lambda x: re.sub("['[\]]", "", x))

In [155]:
df_88R['featured'] = df_88R['featured'].apply(lambda x: x.split(sep=', '))

In [82]:
type(df_88R['featured'][0])

list

In [83]:
df_88R['featured'][0]

['88rising', 'BIBI']

Our goal is to do an analysis on the tracks with collaborations from other artists outside of 88 Rising. Therefore, we will need to split the dataframe into two dataframes (one with no collaborations and another with collaborations).

In [9]:
len(df_88R['featured'][4]) > 1

False

In [10]:
len(df_88R['featured'][0])

2

In [156]:
collabs_df = df_88R[df_88R['featured'].apply(lambda x: len(x) > 1)]
collabs_df = collabs_df.reset_index(drop=True)
collabs_df.head()

,song name,album,artist,featured,release_date,length(ms),popularity,acousticness,danceability,energy,liveness,tempo,valence
0,The Weekend,The Weekend,88rising,"[88rising, BIBI]",2021-10-14,167661,69,0.06200,0.784,0.521,0.0995,101.491,0.817
1,LAW (Prod. Czaer),Street Man Fighter Original Vol.3 (Mission by ...,Various Artists,"[Yoon Mirae, BIBI]",2022-09-06,189373,71,0.00857,0.756,0.900,0.2370,128.054,0.550
2,Best Lover,Head In The Clouds Forever,88rising,"[88rising, BIBI]",2022-04-16,152910,66,0.26800,0.740,0.663,0.1210,82.004,0.519
3,Never Gonna Come Down,Shang-Chi and The Legend of The Ten Rings: The...,88rising,"[Mark Tuan, BIBI]",2021-09-03,203800,51,0.65600,0.777,0.689,0.0620,150.036,0.873
4,Crazy Like You (feat. BIBI),"Bare&Rare, Pt. 1",CHUNG HA,"[CHUNG HA, BIBI]",2022-07-11,175480,50,0.09570,0.536,0.639,0.2090,170.940,0.502


In [167]:
no_collabs_df = df_88R[df_88R['featured'].apply(lambda x: len(x) == 1)]
no_collabs_df = no_collabs_df.reset_index(drop=True)
no_collabs_df.head()

,song name,album,artist,featured,release_date,length(ms),popularity,acousticness,danceability,energy,liveness,tempo,valence
0,Animal Farm,Animal Farm,BIBI,[BIBI],2022-09-27,203945,69,0.0178,0.617,0.570,0.5500,107.971,0.375
1,KAZINO,KAZINO,BIBI,[BIBI],2020-04-29,185946,65,0.1390,0.625,0.686,0.1100,133.041,0.263
2,BAD SAD AND MAD,Life is a Biâ¦,BIBI,[BIBI],2021-04-28,154988,64,0.5450,0.746,0.558,0.0904,90.013,0.638
3,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,[BIBI],2022-02-20,214851,60,0.6780,0.528,0.423,0.1670,135.917,0.249
4,MotoSpeed 24,Sweet Sorrow of Mother,BIBI,[BIBI],2022-10-24,133904,56,0.3190,0.610,0.597,0.1360,95.004,0.295


In [169]:
print(len(collabs_df), len(no_collabs_df))

200 251


Filter data on collaborations with artists outside of 88 Rising.
collab_88 are the list of songs featuring only 88 Rising artists while collab_out are the list of 88 Rising songs with featured artists outside of 88 Rising.

In [162]:
collab_88 = []
collab_out = []

In [163]:
lowercase_artists = [x.lower() for x in artists]

In [134]:
len(collabs_df)

200

In [164]:
for index, row in collabs_df.iterrows():
    for a in row['featured']:
        if a.lower() in lowercase_artists or a.lower() == '88rising' or a.lower() =='chung ha': # Chung Ha is mispelled in tracks csv
            if a == row['featured'][-1]:
                collab_88.append(index)
        else:
            collab_out.append(index)
            break
print(len(collab_88), len(collab_out))

36 164


In [165]:
collab_88_df = collabs_df[collabs_df.index.isin(collab_88)]
collab_88_df.head()

,song name,album,artist,featured,release_date,length(ms),popularity,acousticness,danceability,energy,liveness,tempo,valence
0,The Weekend,The Weekend,88rising,"[88rising, BIBI]",2021-10-14,167661,69,0.062000,0.784,0.521,0.0995,101.491,0.817
2,Best Lover,Head In The Clouds Forever,88rising,"[88rising, BIBI]",2022-04-16,152910,66,0.268000,0.740,0.663,0.1210,82.004,0.519
4,Crazy Like You (feat. BIBI),"Bare&Rare, Pt. 1",CHUNG HA,"[CHUNG HA, BIBI]",2022-07-11,175480,50,0.095700,0.536,0.639,0.2090,170.940,0.502
13,froyo (feat. Warren Hue),Head In The Clouds Forever,88rising,"[88rising, BIBI, Rich Brian]",2022-04-16,309121,45,0.213000,0.508,0.688,0.1570,145.089,0.335
37,These Nights,These Nights,88rising,"[88rising, Rich Brian]",2019-10-03,223006,41,0.000317,0.656,0.673,0.2800,83.018,0.494


In [166]:
collab_out_df = collabs_df[collabs_df.index.isin(collab_out)]
collab_out_df.head()

,song name,album,artist,featured,release_date,length(ms),popularity,acousticness,danceability,energy,liveness,tempo,valence
1,LAW (Prod. Czaer),Street Man Fighter Original Vol.3 (Mission by ...,Various Artists,"[Yoon Mirae, BIBI]",2022-09-06,189373,71,0.00857,0.756,0.900,0.2370,128.054,0.550
3,Never Gonna Come Down,Shang-Chi and The Legend of The Ten Rings: The...,88rising,"[Mark Tuan, BIBI]",2021-09-03,203800,51,0.65600,0.777,0.689,0.0620,150.036,0.873
5,AUTOMATIC,AUTOMATIC,Various Artists,"[Chancellor, Babylon, twlv]",2020-10-14,268293,51,0.13300,0.720,0.703,0.1210,99.999,0.492
6,Eleven (feat. BIBI),Blueline,twlv,"[twlv, BIBI]",2019-03-15,196520,46,0.14500,0.502,0.576,0.1700,89.897,0.205
7,Hanryang,Hanryang,Min Kyunghoon,"[Min Kyunghoon, KIM HEECHUL, BIBI]",2020-12-19,202866,49,0.06050,0.698,0.729,0.0918,140.083,0.306
